

# Exercicio - SVM - Support Vector Machines

Neste tutorial, vamos trabalhar com o classificador SVM, ou Máquina de Vetores de Suporte. Um modelo muito usado em Machine Learning e capaz de lidar com problemas linear e não linearmente separáveis.

Assim como nos exemplos anteriores, vamos nos prender em dataset bi-dimensionais, pois isso nos permitirá uma ampla visualização do que acontece por baixo dos panos nesses classificadores. Poderemos visualizar os pontos no espaço de busca/classificação e as superfícies de decisão. No final do tutorial aplicaremos esses métodos a problemas com mais dimensões. Vocês verão que o código é exatamente o mesmo para estes casos.

Imagine que estejam construindo uma rua em seu bairro e o prefeito quer que ela seja o mais larga possível, de forma a separar as casas do lado direito e esquerdo da rua. O que o SVM busca fazer é muito similar a isso, encontrar as margens mais disnte possíveis entre si que separem os exemplos de cada uma das classes. A reta no centro destas margens (ou seja, a faixa que separa as duas mãos da nossa rua), é a nossa superfície de decisão.

Observe a imagem abaixo, extraída do livro Introduction to Statistical Learning. Neste caso desejamos separar os pontos da classe azul dos pontos da classe roxa. As duas retas pontilhadas são as margens de cada classe (que o SVM busca aprender), enquanto a reta contínua é nossa superície de decisão.

Antes da explicação matemática do SVM, vamos sentir o gostinho de executá-lo com o Scikit Learning utilizando um dataset clássico, o Iris. Neste modelo vamos classificar se uma íris (flor) é do tipo Setosa ou Versicolor

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [ ]:
iris = datasets.load_iris()

In [ ]:
iris = datasets.load_iris()
X = iris["data"]  #dimensoes da petala
y = iris["target"]

#selecionamos apenas os dois tipos que temos interesse


NameError: ignored

In [ ]:
svm_clf = SVC(C=100, kernel='linear')
svm_clf.fit(X, y)

In [ ]:
svm_clf.predict([[2.5, 1.1]])

Vamos visualizar esses dados para entender melhor o comportamento do nosso classificador.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_svc_decision_boundary(svm_clf, xmin, xmax):
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]

    # a reta de nossa superfície de decisão tem como equacao w0*x0 + w1*x1 + b = 0, logo
    # x1 = -w0/w1 * x0 - b/w1
    x0 = np.linspace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1] #todos os valores de x1 em funcao dos valores de x0 na sup de decisao

    margin = 1/w[1] #estamos desenhando a superficie em funcao de x1, logo a margem também sera em funcao dele, por isso divido por 1/w[1], que é o deslocamento da superficie de decisao para as maregsn
    margin_up = decision_boundary + margin
    margin_down = decision_boundary - margin

    plt.plot(x0, decision_boundary, "k-", linewidth=2)
    plt.plot(x0, margin_up, "k--", linewidth=2)
    plt.plot(x0, margin_down, "k--", linewidth=2)

In [ ]:
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris-Versicolor")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris-Setosa")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plot_svc_decision_boundary(svm_clf, 0, 5.5)
plt.axis([0, 5.5, 0, 2])


Note que o nosso modelo conseguiu aprender um hiperplano de separação entre as duas classes, que fica exatamente no centro das duas margens máximas encontradas por ele. Desta forma a, superfície de decisão encontrada é aquela que fica o mais distante possível das duas classes.

Observe que o parâmetro C=100 foi empregado no nosso modelo. Essa parâmtro indica o quão rigoroso eu desejo que o modelo seja em relação a violações à margem, ou seja, a pontos que fiquem do lado oposto à margem encontrada para sua classe. Isto é importante pois em alguns casos pode ser que não seja possível encontrar uma margem que separe todos os pontos, ou, se essa margem existir, ela seja muito pequena, tornando a generalização do modelo para novos exemplos muito ruim. Um outro problema que pode ser causado com um hiperparâmetro muito rigoroso é gerar uma grande sensibilidade a outliers.

Observe como ficaria nosso modelo com um parâmetro C=1

In [ ]:
svm_clf = SVC(C=0.1, kernel='linear')
svm_clf.fit(X, y)

In [ ]:
plt.figure(figsize=(12,2.7))

plt.subplot(121)
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris-Versicolor")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris-Setosa")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plot_svc_decision_boundary(svm_clf, 0, 5.5)
plt.axis([0, 5.5, 0, 2])

Note que neste caso estamos sendo mais permissivos, o que nos gera margens maiores.

Observe o que aconteceria se houvesse um outlier no nosso modelo

In [ ]:
X_outliers = np.array([[3.4, 1.3], [3.2, 0.8]])
y_outliers = np.array([0, 0])
X1_with_outlier = np.concatenate([X, X_outliers[:1]], axis=0)
y1_with_outlier = np.concatenate([y, y_outliers[:1]], axis=0)
X2_with_outlier = np.concatenate([X, X_outliers[1:]], axis=0)
y2_with_outlier = np.concatenate([y, y_outliers[1:]], axis=0)

In [ ]:
svm_clf = SVC(kernel="linear", C=10**9)
svm_clf.fit(X2_with_outlier, y2_with_outlier)

In [ ]:
plt.plot(X2_with_outlier[:, 0][y2_with_outlier==1], X2_with_outlier[:, 1][y2_with_outlier==1], "bs")
plt.plot(X2_with_outlier[:, 0][y2_with_outlier==0], X2_with_outlier[:, 1][y2_with_outlier==0], "yo")
plot_svc_decision_boundary(svm_clf, 0, 5.5)
plt.xlabel("Petal length", fontsize=14)
plt.annotate("Outlier",
             xy=(X_outliers[1][0], X_outliers[1][1]),
             xytext=(3.2, 0.08),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.1),
             fontsize=16,
            )
plt.axis([0, 5.5, 0, 2])

Note como neste caso nossa margem se tornou extramemente pequena devido apenas a um exemplo. Isto pode causar erros de classificação durante o processo de produção, pois o modelo não conseguirá classificar corretamente um ponto da classe azul que esteja mais mais deslocado sentido classe amararela.

Uma outra técnica importante de ser empregada com o SVM é escalar (normalizar subtraindo a média e dividindo pelo desvio padrão) os dados. Este método é muito sensível à escala, uma vez que busca maximizar a margem de separação utilizando uma métrica de distância do ponto à margem, ele provavelmente irá negligenciar features com pequenas dimensões, enquanto features com valores altos dominarão o cálculo da distância. Veja o exemplo abaixo:

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X_new = np.array([[1, 50], [5, 20], [3, 80], [5, 60]]).astype(np.float64)
y_new = np.array([0, 0, 1, 1])

svm_clf2 = SVC(kernel="linear", C=100)
svm_clf2.fit(X_new, y_new)

plt.figure(figsize=(12,3.2))
plt.subplot(121)
plt.plot(X_new[:, 0][y_new==1], X_new[:, 1][y_new==1], "bo")
plt.plot(X_new[:, 0][y_new==0], X_new[:, 1][y_new==0], "rs")
plot_svc_decision_boundary(svm_clf2, 0, 6)
plt.xlabel("$x_0$", fontsize=20)
plt.ylabel("$x_1$  ", fontsize=20, rotation=0)
plt.title("Sem normalização", fontsize=16)
plt.axis([0, 6, 0, 90])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)
svm_clf2.fit(X_scaled, y_new)

plt.subplot(122)
plt.plot(X_scaled[:, 0][y_new==1], X_scaled[:, 1][y_new==1], "bo")
plt.plot(X_scaled[:, 0][y_new==0], X_scaled[:, 1][y_new==0], "rs")
plot_svc_decision_boundary(svm_clf2, -2, 2)
plt.xlabel("$x_0$", fontsize=20)
plt.title("Com normalização", fontsize=16)
plt.axis([-2, 2, -2, 2])

Observe que a escala da feature $x_1$ na imagem da esquerda é muito maior que a da feature $x_0$, e que a margem do modelo sem normalização é significativamente menor (em proporção) e horizontal, uma vez que $x_1$ está dominando o cálculo da distância.Note também que no segundo caso mais pontos participam da margem (são vetores de suporte), já que a feature $x_0$ passa a contribuir igualmente para o cálculo de distânca.

No exemplo utilizando o dataset Iris, acima, não normalizamos o dados apenas para fins didádicos de facilitar a visualização.

<h5>Como o SVM classifica os dados?</h5>

Matematicamente, após treinado o modelo, a tarefa de classificação do SVM é bem simples e segue de acordo com a equação abaixo:

$
\begin{equation}
  y^{(i)}_{predito} ==\left\{
  \begin{array}{@{}ll@{}}
    0, & \text{se}\ w_1x_1 + w_2x_2 + \dots + w_mx_m + b <  0 \\
    1, & \text{se}\ w_1x_1 + w_2x_2 + \dots + w_mx_m + b \geq 0
  \end{array}\right.
\end{equation}
$

Ou, de forma vetorial

$
\begin{equation}
  \hat{y} ==\left\{
  \begin{array}{@{}ll@{}}
    0, & \text{se}\ W^T\cdot  x + b <  0 \\
    1, & \text{se}\ W^T\cdot x + b \geq  0  
  \end{array}\right.
\end{equation}
$

onde $W$ e $b$ são os coeficientes da superfície de decisão, encontrados pelo SVM.

Vamos ver um exemplo para os pontos $x_1 = (1.5, 0.7)$ e $x_2 = (4.1, 1.1)$, em destaque no gráfico seguinte.

In [ ]:
x1 = [1.5, 0.7]
x2 = [4.1, 1.1]

svm_clf = SVC(kernel="linear", C=100)
svm_clf.fit(X, y)

In [ ]:
plt.figure(figsize=(12,2.7))

plt.subplot(121)
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris-Versicolor")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris-Setosa")
plt.scatter(x1[0], x1[1], s=50, alpha=0.5, c="g")
plt.scatter(x2[0], x2[1], s=50, alpha=0.5, c="m")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plot_svc_decision_boundary(svm_clf, 0, 5.5)
plt.axis([0, 5.5, 0, 2])

Vamos implementar manualmente o cálculo de classificação do SVM, porém com o Scikit podemos obter a classe rapidamente utilizando o método predict()

In [ ]:
score = np.dot(svm_clf.coef_, x1) + svm_clf.intercept_
classe = 1 if score > 0 else 0
print("O ponto " + str(x1) + " pertence à classe " + str(classe) + " com score " + str(score[0]))

In [ ]:
score = np.dot(svm_clf.coef_, x2) + svm_clf.intercept_
classe = 1 if score > 0 else 0
print("O ponto " + str(x2) + " pertence à classe " + str(classe) + " com score " + str(score[0]))

Mas matematicamente, o que isto quer dizer?

A nossa função de decisão $h = W^T\cdot x + b$ pode ser visualizada no gráfico abaixo (plano azul hachurado). Quando calculamos o valor de $h$ estamos gerando um score que obedece a regra de classificação do SVM (foi a partir desta regra que ele encontrou os parâmetros $W$) de acordo com as posição do exemplo em questão no espaço multidimensional. Observe que nossa superfície de decisão é o plano onde $h =
0$ e a nosso limite de decisão (decision boundery) é o hiperplano (reta no nosso caso) onde as duas regiões em destaque se cruzam.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def plot_3D_decision_function(ax, w, b, x1_lim=[0, 5.0], x2_lim=[0, 2.0], plot_margin=True):
    x1_in_bounds = (X[:, 0] > x1_lim[0]) & (X[:, 0] < x1_lim[1])
    X_crop = X[x1_in_bounds]
    y_crop = y[x1_in_bounds]
    x1s = np.linspace(x1_lim[0], x1_lim[1], 40)
    x2s = np.linspace(x2_lim[0], x2_lim[1], 40)
    x1, x2 = np.meshgrid(x1s, x2s)
    xs = np.c_[x1.ravel(), x2.ravel()]
    df = (xs.dot(w) + b).reshape(x1.shape)
    m = 1 / np.linalg.norm(w)
    if plot_margin:
        boundary_x2s = -x1s*(w[0]/w[1])-b/w[1]
        margin_x2s_1 = -x1s*(w[0]/w[1])-(b-1)/w[1]
        margin_x2s_2 = -x1s*(w[0]/w[1])-(b+1)/w[1]
        boundary_in_bounds = (boundary_x2s > x1_lim[0]) & (boundary_x2s <= x2_lim[1])
        m1_in_bounds = (margin_x2s_1 > x1_lim[0]) & (margin_x2s_1 < x2_lim[1])
        m2_in_bounds = (margin_x2s_2 > x1_lim[0]) & (margin_x2s_2 < x2_lim[1])
        ax.plot(x1s[boundary_in_bounds], boundary_x2s[boundary_in_bounds], 0, "k-", linewidth=2, label=r"Decisão")
        ax.plot(x1s[m1_in_bounds], margin_x2s_1[m1_in_bounds], 0, "k--", linewidth=2, label=r"Margem")
        ax.plot(x1s[m2_in_bounds], margin_x2s_2[m2_in_bounds], 0, "k--", linewidth=2)
    scores_1 = svm_clf.decision_function(X_crop[y_crop==1])
    scores_2 = svm_clf.decision_function(X_crop[y_crop==0])
    ax.plot_surface(x1, x2, df, alpha=0.1, color="b")
    ax.plot_wireframe(x1, x2, df, alpha=0.05, color="k")
    ax.plot_surface(x1s, x2, np.zeros_like(x1),
                color="g", alpha=0.2)
    ax.plot(X_crop[:, 0][y_crop==1], X_crop[:, 1][y_crop==1], scores_1, "bs")
    ax.plot(X_crop[:, 0][y_crop==0], X_crop[:, 1][y_crop==0], scores_2, "yo")
    ax.axis(x1_lim + x2_lim)
    ax.set_xlabel(r"Petal length", fontsize=15)
    ax.set_ylabel(r"Petal width", fontsize=15)
    ax.set_zlabel(r"$h = \mathbf{w}^T \mathbf{x} + b$", fontsize=18)
    ax.legend(loc="upper left", fontsize=16)


fig = plt.figure(figsize=(20, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax1.elev = 40
plot_3D_decision_function(ax1, w=svm_clf.coef_[0], b=svm_clf.intercept_[0], plot_margin=True)

ax2 = fig.add_subplot(122, projection='3d')
ax2.elev = 10
ax2.azim = 100
plot_3D_decision_function(ax2, w=svm_clf.coef_[0], b=svm_clf.intercept_[0], plot_margin=True)

<h3>Obtendo probabilidades</h3>

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
svm_clf_probs = SVC(kernel="linear", probability=True)
svm_clf_probs.fit(X, y)

In [ ]:
svm_scores = svm_clf_probs.decision_function(X)
svm_scores_expanded = np.expand_dims(svm_scores, axis=1)

In [ ]:
lr = LogisticRegression()
lr.fit(svm_scores_expanded, y)

In [ ]:
lr.predict_proba(svm_scores_expanded)[0:10]

In [ ]:
svm_clf_probs.predict_proba(X)[0:10]

O SVM funciona muito bem em muitos casos, porém quase todos os datasets não são linearmente separáveis. Uma abordagem para lidar com este problema é a utilização de features polinomiais, assim como fazemos com regressão linear (polinomial). Veja o exemplo abaixo. Como separar estes dados utilizando o SVM?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
c1 = np.array([-4, -3, 3, 4])
c2 = np.array([-2, -1, 0, 1, 2])

In [ ]:
plt.scatter(x=c1, y=np.zeros_like(c1), c='blue')
plt.scatter(x=c2, y=np.zeros_like(c2), c='red')
plt.yticks([])

Se aumentarmos uma dimensão no nosso espaço, adicionando uma feature $x_2 = x^2$, projetamos nossos dados em um espaço onde eles são linearmente separáveis.

In [ ]:
c1_2 = c1**2
c2_2 = c2**2

In [ ]:
plt.scatter(x=c1, y=c1_2, c='blue')
plt.scatter(x=c2, y=c2_2, c='red')
plt.axhline(y=6, ls='--', c='gray')

Utilizando o Scikit-Learn conseguimos implementar o SVM com essa característica facilmente, é só adicionarmos o PolynomialFeatures a nosso pipeline:

In [ ]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
polynomial_svm_clf = Pipeline([
    ("poly_fetaures", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])

In [ ]:
polynomial_svm_clf.fit(X, y)

Adicionar features polinomiais funciona muito bem na maioris dos algoritmos de Machine Learning (não apenas não apenas no SVM), porém um polinômio de baixo grau não é capaz de lidar com problemas complexos e polinômios de graus muito altos geram um número muito grande de features, prejudicando a performance do modelo. O svm nos permite aplicar quase todas as técnincas matemáticas sem necessariamente adicionarmos dimensões a nosso espaço, apenas modificando as métricas de distância/similaridade por meio dos <b>kernel tricks</b>, evitando a explosão combinatória do número de features.

In [ ]:
from sklearn.svm import SVC

In [ ]:
poly_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel='poly', degree=3, coef0=1, C=5))
])

In [ ]:
poly_kernel_svm_clf.fit(X, y)

Uma outra forma de lidarmos com problemas não lineares é criarmos features de semelhança computadas por meio de uma função de similaridade. A Gaussian Radial Bases Function (RBF) é um exemplo de função de similaridade :


$\phi\gamma(x, \ell) = exp(-\gamma\|x-\ell\|^2)$

que é uma função gaussiana (simétrica em forma de sino) que assume valores entra 0 e 1

Voltando ao nosso primeiro conjunto de dados:

In [ ]:
x1 = -1

In [ ]:
plt.scatter(x=c1, y=np.zeros_like(c1), c='blue')
plt.scatter(x=[-1], y=[0], s=150, alpha=0.5, c="m")
plt.scatter(x=c2, y=np.zeros_like(c2), c='red')
plt.yticks([])

Vamos analisar este exemplo em relação a dois pontos de referência, $\ell_1 = -2$ e $\ell_2 = 1$. O exemplo $x_1 = -1$ (em destaque) está originalmente localizado a uma distância 1 da primeira referência e 2 da segunda. Porém suas novas features são $x_{1,1} =exp(-0.3\times 1^2) \approx 0.74$ e $x_{1,2} =exp(-0.3\times 2^2) \approx 0.30$ Como pode ser visto no gráfico abaixo, com isso elas são linearmente separáveis.

In [ ]:
def gaussian_rbf(x, ell, gamma=0.3):
    return np.exp(-gamma * np.linalg.norm(x-ell)**2)

In [ ]:
c1_g1 = np.array([gaussian_rbf(x,-2) for x in c1])
c1_g2 = np.array([gaussian_rbf(x, 1) for x in c1])
c2_g1 = np.array([gaussian_rbf(x,-2) for x in c2])
c2_g2 = np.array([gaussian_rbf(x, 1) for x in c2])

In [ ]:
x1_g = gaussian_rbf(x1, -2)
x2_g = gaussian_rbf(x1, 1)

In [ ]:
plt.scatter(x=c1_g1, y=c1_g2, c='blue')
plt.scatter(x=[x1_g], y=[x2_g], s=150, alpha=0.5, c="m")
plt.scatter(x=c2_g1, y=c2_g2, c='red')
x1, y1 = [-0.1, 1.1], [0.57, -0.1]
plt.plot(x1, y1, c='gray', ls='--')
plt.axis([-0.1, 1.1, -0.1, 1.1])

Note que com a utilização de kernels não adicionamos dimensões a nosso espaço, mas sim computamos novas features de acordo com uma função de kernel.

Observe que agora as amostras são linearmente separáveis. Neste caso nos perguntamos como escolher os pontos de referência. Geralmente esses pontos são calculados como todos os pontos do dataset, ou seja, cada exemplo se torna um ponto de referência e calculamos a RBF para todos.

In [ ]:
rbf_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel='rbf', C=5))
])
rbf_kernel_svm_clf.fit(X, y)

É importante destacar que quando utilizamos as funções de kernel nós não adicionamos features ao nosso conjunto de dados, mas sim calculamos uma distância "similuando" que estes dados tenham sido mapeados a num novo espaço. O gráfico acima apenas ilustra a distancia entre os pontos utilizando o kernel rbf em relação aos pontos de referência $\ell_1$ e $\ell_2$.

<h3>SVM Regression</h3>

Também é possível utilizarmos o SVM para problemas de regressão. Neste caso, o SVM busca coeficientes que minimizem uma loss onde apenas os resíduos mair que uma contante pré definida contribuiem par a funçãso de loss.

In [ ]:
from sklearn.svm import LinearSVR

In [ ]:
svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

Assim como na regressão polinomial, podemos lidar com problemas não lineares pelo SVR, apenas se faz necessário o uso de kernels.

In [ ]:
from sklearn.svm import SVR

In [ ]:
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_reg.fit(X,y)